# Planning Notebook

In [77]:
import math
import unittest
import numpy as np
from itertools import product
import tqdm
from tqdm import tqdm_notebook
import copy
import pickle
import time

# import gtsam
import gtsam
from gtsam import *
from gtsam.utils.test_case import GtsamTestCase

# import gtbook
import gtbook
from gtbook.display import *
from gtbook.discrete import *

# import local package
import gtsam_planner
from gtsam_planner import *

# import parser
import SASParser
from SASParser import SAS, Operator

import SasToGtsam
from SasToGtsam import SASToGTSAM

variables = Variables()
def pretty(obj): 
    return gtbook.display.pretty(obj, variables)

import graphviz
class show(graphviz.Source):
    """ Display an object with a dot method as a graph."""

    def __init__(self, obj):
        """Construct from object with 'dot' method."""
        # This small class takes an object, calls its dot function, and uses the
        # resulting string to initialize a graphviz.Source instance. This in turn
        # has a _repr_mimebundle_ method, which then renders it in the notebook.
        super().__init__(obj.dot())

In [78]:
sas = SAS()
sas_dir = "sas/block_example.sas"
sas.read_file(sas_dir)
converter = SASToGTSAM(sas)

In [79]:
# def plan(plan_length):
#     for k in range(2, plan_length):
#         print(k)
#         states = []
#         mutex_factors = []
#         op_factors = []

#         for i in range(k):
#             # generate state
#             state_t = converter.generate_state(i)
#             states.append(state_t)
#             # generate mutex factor for the state
#             mutex_factor_t = converter.generate_mutex_factor(state_t)
#             mutex_factors.append(mutex_factor_t)
#         for j in range(len(states)-1):
#             op_key = converter.generate_operator_key(j)
#             op_factor = converter.generate_op_factor(states[j], states[j+1], op_key)
#             op_factors.append(op_factor)
#         initial_factor = converter.generate_initial_factor(states[0])
#         goal_factor = converter.generate_goal_factor(states[-1])

#         graph = gtsam.DiscreteFactorGraph()
#         for m_factor in mutex_factors:
#             for f in m_factor:
#                 graph.push_back(f)

#         graph.push_back(goal_factor)
#         graph.push_back(initial_factor)

#         for op_factor in op_factors:
#             graph.push_back(op_factor)

#         val = graph.optimize()
#         if graph(val) == 0.0:
#             del graph
#             continue
#         else:
#             return graph, val, k
#     return "longer plan length?"

In [80]:
start = time.time()
k = 7
states = []
mutex_factors = []
op_factors = []
frame_factors = []
for i in range(k):
    # generate state
    state_t = converter.generate_state(i)
    states.append(state_t)
    # generate mutex factor for the state
    mutex_factor_t = converter.generate_mutex_factor(state_t)
    mutex_factors.append(mutex_factor_t)
for j in range(len(states)-1):
    op_key = converter.generate_operator_key(j)
    op_factor, frame_factor = converter.generate_frame_op_factor(states[j], states[j+1], op_key)
    op_factors.append(op_factor)
    frame_factors.append(frame_factor)
initial_factor = converter.generate_initial_factor(states[0])
goal_factor = converter.generate_goal_factor(states[-1])

graph = gtsam.DiscreteFactorGraph()
for m_factor in mutex_factors:
    for f in m_factor:
        graph.push_back(f)

graph.push_back(goal_factor)
graph.push_back(initial_factor)

for op_factor in op_factors:
    graph.push_back(op_factor)

for frame_factor in frame_factors:
    graph.push_back(frame_factor)

val = graph.optimize()
end = time.time()
assert graph(val) == 1
print("time taken:", end - start)

time taken: 59.52446484565735


In [81]:
# graph, val, k = plan(12)

In [82]:
print(graph)


size: 49
factor 0: MutexConstraint on 139 138 144 145 146 
factor 1: MutexConstraint on 140 138 144 145 146 
factor 2: MutexConstraint on 141 138 144 145 146 
factor 3: MutexConstraint on 142 138 144 145 146 
factor 4: MutexConstraint on 143 138 144 145 146 
factor 5: MutexConstraint on 148 147 153 154 155 
factor 6: MutexConstraint on 149 147 153 154 155 
factor 7: MutexConstraint on 150 147 153 154 155 
factor 8: MutexConstraint on 151 147 153 154 155 
factor 9: MutexConstraint on 152 147 153 154 155 
factor 10: MutexConstraint on 157 156 162 163 164 
factor 11: MutexConstraint on 158 156 162 163 164 
factor 12: MutexConstraint on 159 156 162 163 164 
factor 13: MutexConstraint on 160 156 162 163 164 
factor 14: MutexConstraint on 161 156 162 163 164 
factor 15: MutexConstraint on 166 165 171 172 173 
factor 16: MutexConstraint on 167 165 171 172 173 
factor 17: MutexConstraint on 168 165 171 172 173 
factor 18: MutexConstraint on 169 165 171 172 173 
factor 19: MutexConstraint on 1

In [83]:
graph(val)

1.0

In [84]:
op_consts = []
for i in range(graph.size()-1, graph.size()-k, -1):
    op_consts.append(graph.at(i))
val_list = []
for op_const in reversed(op_consts):
    print(converter.ops_names[val[op_const.operatorKey()]])

pick-up b
stack b a
pick-up c
stack c b
pick-up d
stack d c
